In [1]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver
from datetime import datetime, timedelta
from pytimekr import pytimekr # 공휴일 반환
from haversine import haversine
from datetime import date
import time
import json
import requests
import sys
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Button, Layout, HBox, AppLayout
import warnings
warnings.filterwarnings(action='ignore')

---

**<center style='font-size:50px'><span style='color:blue'>주차장 추천 알고리즘</p>**

---

In [2]:
def addr_to_lat_lon(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={address}'.format(address=addr)
    headers = {"Authorization": "KakaoAK " + '9b9f200ef72a1763c56ad4540d89dc53'}
    result = json.loads(str(requests.get(url, headers=headers).text))
    match_first = result['documents'][0]['address']
    return float(match_first['y']),float(match_first['x']) # 위도, 경도 순

In [3]:
def operation(in_time, out_time, data):
    in_time_weekday = in_time.weekday()      # 주차 요일
    out_time_weekday = out_time.weekday()    # 출차 요일
    
    kr_holidays = pytimekr.holidays(year=datetime.now().year) # 공휴일 날짜
    in_time_date = in_time.date()                             # 주차 날짜
    out_time_date = out_time.date()                           # 출차 날짜
    
    park_in_time = in_time.strftime("%H%M")     # 주차 시간
    park_out_time = out_time.strftime("%H%M")   # 출차 시간

    
    weekday = [0, 1, 2, 3, 4]
    weekend = [5]
    holiday = [6]
    
    in_time_pass = []   # 주차 시간에 운영하는 주차장
    out_time_pass = []  # 출차 시간에 운영하는 주차장
    
    for i in range(len(data)):
        if (in_time_weekday in weekday) and (in_time_date not in kr_holidays):
            if int(park_in_time) >= data['평일 운영 시작시각(HHMM)'][i]:
                in_time_pass.append(i)
        elif (in_time_weekday in weekend) and (in_time_date not in kr_holidays):
            if int(park_in_time) >= data['주말 운영 시작시각(HHMM)'][i]:
                in_time_pass.append(i)
        elif (in_time_weekday in holiday) or (in_time_date in kr_holidays):
            if int(park_in_time) >= data['공휴일 운영 시작시각(HHMM)'][i]:
                in_time_pass.append(i)
                
        if (out_time_weekday in weekday) and (out_time_date not in kr_holidays):
            if int(park_out_time) <= data['평일 운영 종료시각(HHMM)'][i]:
                out_time_pass.append(i)
        elif (out_time_weekday in weekend) and (out_time_date not in kr_holidays):
            if int(park_out_time) <= data['주말 운영 종료시각(HHMM)'][i]:
                out_time_pass.append(i)
        elif (out_time_weekday in weekend) or (out_time_date in kr_holidays):
            if int(park_out_time) <= data['공휴일 운영 종료시각(HHMM)'][i]:
                out_time_pass.append(i)
                
    inter_tuple = set(in_time_pass).intersection(out_time_pass) # 주차시간, 출차시간 모두 운영하는 주차장
    intersection = list(inter_tuple)
    return intersection

In [4]:
def date_range(start, end):
    start = datetime.strptime(f'{start.year}-{start.month}-{start.day}', "%Y-%m-%d")
    end = datetime.strptime(f'{end.year}-{end.month}-{end.day}', "%Y-%m-%d")
    dates = [(start + timedelta(days=i)).date() for i in range((end-start).days+1)]
    return dates

In [5]:
def checking_free(week, date, holidy_check, weekend_check):
    kr_holidays = pytimekr.holidays(year=datetime.now().year)
    
    weekday = [0, 1, 2, 3, 4]
    weekend = [5]
    holiday = [6]           
                    
    if ((week in holiday) and holidy_check == '무료') or ((week in weekend) and weekend_check == '무료') or ((date in kr_holidays) and holidy_check == '무료'):
        check = True
    else:
        check = False
        
    return check

In [6]:
def fee_calcultaion(in_time, out_time, fee_data):
    all_date = date_range(in_time, out_time)
    between_date = all_date[1:-1]
    
    in_time_date, in_time_week = all_date[0], all_date[0].weekday()
    out_time_date, out_time_week = all_date[-1], all_date[-1].weekday()
    kr_holidays = pytimekr.holidays(year=datetime.now().year)
    
    weekday = [0, 1, 2, 3, 4]
    weekend = [5]
    holiday = [6]
    
    min_diff = int((out_time-in_time) / timedelta(minutes=1))
    day_diff = out_time.day-in_time.day
    
    
    # day 차이가 1 이상일 경우
    full_time = (day_diff-1) * 1440
    in_time_diff = (24-in_time.hour-1)*60 + (60-in_time.minute)
    out_time_diff = out_time.hour*60 + out_time.minute
    
    paid_or_free = fee_data['유무료구분명']
    weekend_check = fee_data['토요일 유,무료 구분명']
    holidy_check = fee_data['공휴일 유,무료 구분명']
    day_max_fee = fee_data['일 최대 요금']
    base_fee = fee_data['기본 주차 요금']
    base_min = fee_data['기본 주차 시간(분 단위)']
    extra_fee = fee_data['추가 단위 요금']
    extra_min = fee_data['추가 단위 시간(분 단위)']
    
    final_fee = 0
    in_day_fee = 0
    out_day_fee = 0
    between_day_fee = 0
    
    if paid_or_free == '무료':
        final_fee = 0 # 최종지출금액
    else: 
        if day_max_fee > 0: # 일 최대요금이 존재하는 경우
            if day_diff == 0:  # 주차 날짜와 출차 날짜가 동일한 경우
                if checking_free(in_time_week, in_time_date, holidy_check, weekend_check) == True:
                    final_fee = 0
                else:
                    final_fee = min(base_fee + (max((min_diff-base_min),0) / extra_min) * extra_fee, day_max_fee)
     
            elif day_diff == 1: # 주차 날짜와 출차 날짜가 동일하지 않을 경우(차이=1)
                if checking_free(in_time_week, in_time_date, holidy_check, weekend_check) == True:
                    if checking_free(out_time_week, out_time_date, holidy_check, weekend_check) == True:
                        out_day_fee = 0
                    else:
                        out_day_fee = min(base_fee + (max((out_time_diff - base_min),0) / extra_min) * extra_fee, day_max_fee)
                else:
                    in_day_fee = min(base_fee + (max((in_time_diff - base_min),0) / extra_min) * extra_fee, day_max_fee)
                    if checking_free(out_time_week, out_time_date, holidy_check, weekend_check) == True:
                        out_day_fee = 0
                    else:
                        out_day_fee = min(base_fee + (max((out_time_diff - base_min),0) / extra_min) * extra_fee, day_max_fee)
            else: # 주차 날짜와 출차 날짜가 동일하지 않을 경우(차이>1)
                between_pay = []
                if checking_free(in_time_week, in_time_date, holidy_check, weekend_check) == True:
                    for i in range(len(between_date)):
                        if checking_free(between_date[i].weekday(), between_date[i], holidy_check, weekend_check) == True:
                            pass
                        else:
                            between_pay.append(i)
                        between_day_fee = len(between_pay) * day_max_fee
                    
                    if checking_free(out_time_week, out_time_date, holidy_check, weekend_check) == True:
                        out_day_fee = 0
                    else:
                        out_day_fee = min(base_fee + (max((out_time_diff - base_min),0) / extra_min) * extra_fee, day_max_fee)
                else:
                    in_day_fee = min(base_fee + (max((in_time_diff - base_min),0) / extra_min) * extra_fee, day_max_fee)
                    for i in range(len(between_date)):
                        if checking_free(between_date[i].weekday(), between_date[i], holidy_check, weekend_check) == True:
                            pass
                        else:
                            between_pay.append(i)
                        between_day_fee = len(between_pay) * day_max_fee
                        
                    if checking_free(out_time_week, out_time_date, holidy_check, weekend_check) == True:
                        out_day_fee = 0
                    else:
                        out_day_fee = min(base_fee + (max((out_time_diff - base_min),0) / extra_min) * extra_fee, day_max_fee)           
        else: # 일 최대요금이 존재하지 않는 경우
            if day_diff == 0:  # 주차 날짜와 출차 날짜가 동일한 경우
                if checking_free(in_time_week, in_time_date, holidy_check, weekend_check) == True:
                    final_fee = 0
                else:
                    final_fee = max(base_fee + (min_diff / extra_min) * extra_fee, day_max_fee) 
            elif day_diff == 1: # 주차 날짜와 출차 날짜가 동일하지 않을 경우(차이=1)
                if checking_free(in_time_week, in_time_date, holidy_check, weekend_check) == True:
                    if checking_free(out_time_week, out_time_date, holidy_check, weekend_check) == True:
                        out_day_fee = 0
                    else:
                        out_day_fee = max(base_fee + (max((out_time_diff - base_min),0) / extra_min) * extra_fee, day_max_fee)
                else:
                    in_day_fee = max(base_fee + (max((in_time_diff - base_min),0) / extra_min) * extra_fee, day_max_fee)
                    if checking_free(out_time_week, out_time_date, holidy_check, weekend_check) == True:
                        out_day_fee = 0
                    else:
                        out_day_fee = max(base_fee + (max((out_time_diff - base_min),0) / extra_min) * extra_fee, day_max_fee)
            else: # 주차 날짜와 출차 날짜가 동일하지 않을 경우(차이>1)
                between_pay = []
                if checking_free(in_time_week, in_time_date, holidy_check, weekend_check) == True:
                    for i in range(len(between_date)):
                        if checking_free(between_date[i].weekday(), between_date[i], holidy_check, weekend_check) == True:
                            pass
                        else:
                            between_pay.append(i)
                        between_day_fee = len(between_pay) * day_max_fee
                    
                    if checking_free(out_time_week, out_time_date, holidy_check, weekend_check) == True:
                        out_day_fee = 0
                    else:
                        out_day_fee = max(base_fee + (max((out_time_diff - base_min),0) / extra_min) * extra_fee, day_max_fee)
                else:
                    in_day_fee = max(base_fee + (max((in_time_diff - base_min),0) / extra_min) * extra_fee, day_max_fee)
                    for i in range(len(between_date)):
                        if checking_free(between_date[i].weekday(), between_date[i], holidy_check, weekend_check) == True:
                            pass
                        else:
                            between_pay.append(i)
                        between_day_fee = len(between_pay) * day_max_fee
                        
                    if checking_free(out_time_week, out_time_date, holidy_check, weekend_check) == True:
                        out_day_fee = 0
                    else:
                        out_day_fee = max(base_fee + (max((out_time_diff - base_min),0) / extra_min) * extra_fee, day_max_fee)

        # 최종 계산
        final_fee = in_day_fee + between_day_fee + out_day_fee
    return final_fee

In [7]:
def parking_lot_recommendation(data, dataset, current_location, Street_name_address, N, park_in, park_out):
    # 도로명 주소 입력단계 / 올바르게 입력하지 않을 시 재 입력 요청
    operation_list = operation(park_in, park_out, dataset)

    data = data.iloc[operation_list].reset_index(drop=True)
    dataset_op = dataset.iloc[operation_list].reset_index(drop=True)
    # 현재 위치를 기준으로 가까운 주차장 거리 계산
    distance = []

    for i in range(len(data)):
        target_location = (data.iloc[i,2], data.iloc[i,3])
        distance.append(haversine(current_location, target_location, unit='km'))
    
    data['거리'] = distance
    
    # 가장 가까운 5개의 주차장에 대한 추가 정보
    top_n = data.sort_values(by='거리', ascending = True).head(N)
    nearlist_index = top_n.index
    
    car_distance = []
    car_time = []
    paking_space = []

    path = 'C:/Users/kijung/Desktop/chromedriver_win32/chromedriver'
    start = time.time()
    
    # Kakao Map 웹 크롤링
    driver = webdriver.Chrome(path)
    driver.get(f'https://map.kakao.com/')    

    for j in range(len(top_n)):
        element = driver.find_element(By.XPATH, '//*[@id="search.tab2"]/a')
        element.send_keys("\n")
        time.sleep(0.1)
        
        element1 = driver.find_element(By.XPATH, '//*[@id="info.route.searchBox.clearVia"]')
        element1.send_keys("\n")
        time.sleep(0.1)
        
        element2 = driver.find_element(By.ID, 'info.route.waypointSuggest.input0')
        element2.send_keys(f'{Street_name_address}')
        element2.send_keys("\n")
        time.sleep(0.1)
        
        element3 = driver.find_element(By.ID, 'info.route.waypointSuggest.input1')
        element3.send_keys(f'{top_n.iloc[j,1]}')
        element3.send_keys("\n")
        time.sleep(0.1)
        
        element4 = element.find_element(By.XPATH, '//*[@id="cartab"]')
        element4.send_keys("\n")
        time.sleep(1)
        
        info_road = element4.find_element(By.XPATH, '//*[@id="info.flagsearch"]/div[6]/ul')
        time.sleep(1)
        info = info_road.text
        information = info
      
        try:
            road_time, road_km = information.split('\n')[0:2]
            car_distance.append(float(road_km[:-2]))
            car_time.append(road_time)
        except:
            road_time = 0
            road_km = 0
            car_distance.append(road_km)
            car_time.append(road_time)
    
    driver.close()
        
    # 서울시 주차정보안내시스템 웹 크롤링        
    driver = webdriver.Chrome(path)
    driver.get("https://parking.seoul.go.kr/")
    for k in range(len(top_n)):
        element = driver.find_element(By.ID, 'input_search2')
        element.send_keys(f'{top_n.iloc[k,0]}')
        element.send_keys("\n")
        time.sleep(0.1)
        
        info_station = element.find_element(By.XPATH, '//*[@id="pk_search_view"]/ul/li/span[1]')     
        info2 = info_station.text
        
        element.clear()
        
        if info2.rfind('주차가능면') == -1:
            space = '확인불가'
        else:
            space = info2[info2.rfind('주차가능면'):]
            
        paking_space.append(space)
        
    driver.close()
    
    end = time.time()
    
    top_n['이동거리(km)'] = car_distance
    top_n['이동시간'] = car_time
    top_n['남은 주차공간'] = paking_space

    # 주차요금 계산
    fee_columns = ['주차장명', '유무료구분명', '토요일 유,무료 구분명', '공휴일 유,무료 구분명', '기본 주차 요금', 
                   '기본 주차 시간(분 단위)', '추가 단위 요금','추가 단위 시간(분 단위)', '일 최대 요금']
    data_fee = pd.DataFrame(dataset_op, columns=fee_columns)
    data_matching = pd.merge(top_n, data_fee, how='inner', on='주차장명').fillna(0).reset_index(drop=True)
    
    fee = []
    
    for k in fee_columns[4:]:
        data_matching[k] = data_matching[k].astype(int)
        
    for l in range(len(data_matching)):
        fee.append(f'{int(fee_calcultaion(park_in, park_out, data_matching.iloc[l]))}'+'원')
    
    top_n['요금'] = fee
    top_n = top_n.sort_values(by='이동거리(km)', ascending = True)
    
    extra_columns = ['주차장명', '주차장 종류명', '운영구분명', '전화번호']
    data_extra = pd.DataFrame(dataset_op, columns=extra_columns)
    data_fin = pd.merge(top_n, data_extra, how='inner', on='주차장명')
    
    data_fin = data_fin.drop(['주차장 위치 좌표 위도', '주차장 위치 좌표 경도'],axis=1)
    print(f'searching 소요 시간 : {end-start:.2f}')
    print('현재 위치에서 가장 가까운 주차장은 다음과 같습니다.')
    data_fin = data_fin.sort_values(by='이동거리(km)', ascending = True)
    return data_fin

In [19]:
style = {'description_width': 'initial'}

# 버튼 위젯
n_select_button = widgets.Button(description = '확인', layout=Layout(width='10%', height='50px'),
                                             style=dict(font_size="15px", button_color = 'lightgreen', font_weight='bold', text_color='red'))
park_button = widgets.Button(description = '확인', layout=Layout(width='10%', height='50px'),
                                             style=dict(font_size="15px", button_color = 'lightgreen', font_weight='bold', text_color='red'))
loc_button = widgets.Button(description = '시작', layout=Layout(width='10%', height='50px'),
                                             style=dict(font_size="15px", button_color = 'red', font_weight='bold'))
output = widgets.Output()


# 날짜 선택 위젯
parking_date = widgets.NaiveDatetimePicker(layout=Layout(width='40%', height='40px'), style=dict(font_size="40x"))
get_out_date = widgets.NaiveDatetimePicker(layout=Layout(width='40%', height='60px'), style=dict(font_size="40px"))

# 추천받을 주차장 수 위젯
show_top_k = widgets.Dropdown(
    options=[1,2,3,4,5,6,7,8,9,10],
    value=1,
    layout=Layout(width='20%', height='30px'),
    style = dict(text_size="25pt", font_size='25pt'),
    diabled=True
)

# 현재 위치 입력 위젯
street_name_address = widgets.Textarea(
    value = "",
    layout=Layout(width='60%', height='90pt'),
    style=dict(
        font_size="25pt",
        background="lightblue",
        description_width='initial'),
    diabled=False)

def n_select_button_clicked(n_select_button):
    with output:
        print('\n')
        print('\n')
        display(widgets.HBox([widgets.Label(value="주차 날짜 및 시간 :", style=dict(font_size='18pt', font_weight='bold')), parking_date]))
        display(widgets.HBox([widgets.Label(value="출차 날짜 및 시간 :", style=dict(font_size='18pt', font_weight='bold')), get_out_date]))
        display(park_button)
        
def park_button_clicked(park_button):
    with output:
        try:
            if parking_date.value <= get_out_date.value:
                print('\n')
                print('\n')
                display(widgets.HBox([widgets.Label(value="현재 위치의 도로명 주소 :", style=dict(font_size='15pt', font_weight='bold')), street_name_address]))
                display(loc_button)
            else:
                print('출차 날짜 또는 시간이 올바르지 않습니다. 다시 입력해주세요')
        except:
            print('제대로 입력해주세요.')
            
def loc_button_clicked(loc_button):
    with output:
        try :
            addr_to_lat_lon(street_name_address.value)
            print('\n')
            print('searching 중입니다....')
            print('\n')
        except:
            print('도로명 주소가 올바르지 않습니다. 다시 입력해주세요')
    
        print(parking_lot_recommendation(data, dataset, addr_to_lat_lon(street_name_address.value), street_name_address.value, show_top_k.value, parking_date.value, get_out_date.value))


dataset = pd.read_csv('서울시 공영주차장 안내 정보.csv', encoding ='euc-kr')
new_index= [14332, 13010, 15627]

data = dataset.drop(index = new_index).drop_duplicates('주차장명').reset_index(drop=True)
dataset = dataset.drop(index = new_index).drop_duplicates('주차장명').reset_index(drop=True)
data = pd.DataFrame(data, columns = ['주차장명', '주소', '주차장 위치 좌표 위도', '주차장 위치 좌표 경도'])

display(widgets.HBox([widgets.Label(value="추천받고 싶은 주차장 수 : ", style=dict(font_size='18pt', font_weight='bold')), show_top_k]))
display(n_select_button, output)


n_select_button.on_click(n_select_button_clicked)
park_button.on_click(park_button_clicked)
loc_button.on_click(loc_button_clicked)

Button(description='확인', layout=Layout(height='50px', width='10%'), style=ButtonStyle(button_color='lightgreen…

Output()